In [ ]:
import mysql.connector
import requests
from mysql.connector import Error
from datetime import datetime
import json
connection = mysql.connector.connect(host='localhost',
                                         database='scrapbms',
                                         user='karthi',
                                         password='password')

cursor = connection.cursor(dictionary=True)
query="select * from districts"
cursor.execute(query)

districts=cursor.fetchall()

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
options=Options()

driver = webdriver.Firefox(options=options)
movie_links=[]
show_links=[]
explore_link='https://in.bookmyshow.com/explore/movies-'
cursor = connection.cursor(dictionary=True)
for row in districts:
    driver.get(explore_link+row['link'])
    soup=BeautifulSoup(driver.page_source)
    shows=soup.find_all("a", {"class": "sc-133848s-11 sc-1ljcxl3-1 eQiiBj"})
    for i in shows[1:]:
        movie_name=i.get("href").split('https://in.bookmyshow.com/',1)[1].split('/')[2]
        movie_id=i.get("href").split('https://in.bookmyshow.com/',1)[1].split('/')[3]
        query="insert into movies(movie_id,movie_name) values(\'{}\',\'{}\');".format(movie_id,movie_name)
        try:
            cursor.execute(query)
        except Error:
            print("movie already added")
        movie_links.append(query)
        link=i.get("href")
        driver.get(link)
        buttons=driver.find_elements(by=By.ID,value="page-cta-container")
        book_ticket=buttons[0]
        if(book_ticket is None):
            print(i.get("href"))
            continue
        book_ticket.click()
        soup=BeautifulSoup(driver.page_source)
        show_times=soup.find_all("a",{"class":"showtime-pill"})
        for j in show_times:
            show_link=j.get("href")
            if(show_link is None):
                continue
            cut_off_time=show_times[0].get('data-cut-off-date-time')
            if(cut_off_time is None):
                continue
            show_id=show_link.split('/booktickets/',1)[1]
            vid=show_id.split('/')[0]
            ssid=show_id.split('/')[1]
            now = datetime.now()
            now_str = now.strftime("%Y%m%d%H%M")
            if(now_str>cut_off_time):
                continue
            txt3 = "https://in.bookmyshow.com/serv/getData?cmd=GETSHOWINFOJSON&vid={}&ssid={}&format=json".format(vid,ssid) 
            response=requests.get(txt3)
            query="insert into shows(dist_id,movie_id,show_id,cutofftime,json_data) values({},\'{}\',\'{}\',\'{}\',\'{}\');".format(row['id'],movie_id,show_id,cut_off_time,response.text)
            try:
                cursor.execute(query)
            except Error:
                print("show already present. updating it")
                query="update shows set json_data=\'{}\' where show_id=\'{}\';".format(response.text,show_id)
                try:
                    cursor.execute(query)
                except:
                    print('update show json failed')
    connection.commit()